In [ ]:
import os 
import numpy as np
import cv2
from glob import glob
from PIL import Image
from keras import backend as K
import imutils
import gc
from sklearn.utils import shuffle
from os.path import join
from keras.utils.np_utils import *
from sklearn import preprocessing
import keras
import random
import math
import pickle
minibatch_size = 20

In [ ]:
# 斷詞
vocab_200 = open('./split_data/words.txt', encoding='UTF-8-sig')
split_word = []
for line in vocab_200:
    line = line.strip('\n')
    split_word.append(line)

In [ ]:
# 做字典
from keras.preprocessing.text import Tokenizer

tok_path = join('split_data', 'vocabulary_tok.pickle')

# saving
if not os.path.exists(tok_path):
    tok = Tokenizer(char_level=False)
    tok.fit_on_texts(split_word)
    with open(tok_path, 'wb') as handle:
        pickle.dump(tok, handle, protocol = pickle.HIGHEST_PROTOCOL)
        print('create tok')
# loading
else:
    with open(tok_path, 'rb') as handle:
        tok = pickle.load(handle)
        print('load tok')

In [ ]:
print(len(tok.word_index))          # 詞彙的個數    1~200

In [ ]:
# for ii,iterm in enumerate(tok.word_index.items()):
#     print(iterm)

In [ ]:
def labels_to_text(label):                       # label is list
    words = tok.sequences_to_texts([[label[0]+1]])
    text = words[0]
    return(text)

def text_to_labels(text):                        # text is string
    seq = tok.texts_to_sequences([text])
    seq = seq[0][0] - 1
    return(seq)

# image

In [ ]:
image_train = np.load('./split_data/image_cordinate/image_bright_train.npy')
image_val = np.load('./split_data/image_cordinate/image_bright_val.npy')

f_val = open('./split_data/image_cordinate/txt_bright_cord_val.txt', encoding='UTF-8-sig')
txt_val = []
for line in f_val:
    line = line.strip('\n')
    txt_val.append(line)
    
f_train = open('./split_data/image_cordinate/txt_bright_cord_train.txt', encoding='UTF-8-sig')
txt_train = []
for line in f_train:
    line = line.strip('\n')
    txt_train.append(line)
    
txt_train = np.array(txt_train)
txt_val = np.array(txt_val)

print(image_train.shape, len(txt_train))
print(image_val.shape, len(txt_val))

In [ ]:
# image_train = image_train.astype('float32')
# image_val = image_val.astype('float32') 

image_train /= 255
image_val /= 255

In [ ]:
label_train = []
for index1 in txt_train:
    temp = text_to_labels(index1)
    label_train.append(temp)
label_train = np.array(label_train)
label_train = to_categorical(label_train, 200)

label_val = []
for index1 in txt_val:
    temp = text_to_labels(index1)
    label_val.append(temp)
label_val = np.array(label_val)
label_val = to_categorical(label_val, 200)

# cordinate

In [ ]:
cord_train = np.load('./split_data/image_cordinate/cord_train.npy')
cord_val = np.load('./split_data/image_cordinate/cord_val.npy')

print(cord_train.shape)
print(cord_val.shape)

In [ ]:
# 計算兩張img的座標差
Subtraction = []
for oneData in cord_train:
    for num in range(len(oneData)-1):
        num1 = num + 1
        sub = oneData[num1] - oneData[num]
        Subtraction.append(sub)
cord_train = np.array(Subtraction).reshape((-1, 76, 20, 2))

Subtraction = []
for oneData in cord_val:
    for num in range(len(oneData)-1):
        num1 = num + 1
        sub = oneData[num1] - oneData[num]
        Subtraction.append(sub)
cord_val = np.array(Subtraction).reshape((-1, 76, 20, 2))

print(cord_train.shape)
print(cord_val.shape)

In [ ]:
# dim 4 to 3
cord_train = cord_train.reshape((len(cord_train), 76, -1))
cord_val = cord_val.reshape((len(cord_val), 76, -1))
print(cord_train.shape)
print(cord_val.shape)

In [ ]:
def Z_ScoreNormalization(data):
    z_score = []
    z_scaler = preprocessing.StandardScaler()
    for i in data:
        iris_z = z_scaler.fit_transform(i)
        z_score.append(iris_z)
    return(np.array(z_score))

cord_train = Z_ScoreNormalization(cord_train)
cord_val = Z_ScoreNormalization(cord_val)
print(cord_train.shape)
print(cord_val.shape)

In [ ]:
cord_train = cord_train.astype('float32')
cord_val = cord_val.astype('float32') 

# image_train /= 255
# image_val /= 255

# model

In [ ]:
from keras import models, layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, SpatialDropout3D
from keras.layers import Convolution3D, MaxPooling3D, LSTM, Conv2D, AveragePooling3D
from keras.layers.convolutional import Conv3D, ZeroPadding3D
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import GRU
from keras.layers.core import Lambda
from keras.layers import Input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from multiprocessing import set_start_method, Pool
set_start_method('forkserver')

In [ ]:
#Define convolution with batchnromalization
def Conv3d_BN(x, nb_filter,kernel_size, padding='same',strides=(1,1,1),name=None):
    x = Conv3D(nb_filter,kernel_size,padding=padding,strides=strides,activation='relu')(x)
    x = BatchNormalization()(x)
    return x

#Define Residual Block for ResNet34(2 convolution layers)
def Residual_Block(input_model,nb_filter,kernel_size,strides=(1,1,1), with_conv_shortcut =False):
    x = Conv3d_BN(input_model,nb_filter=nb_filter,kernel_size=kernel_size,strides=strides,padding='same')
    x = Conv3d_BN(x, nb_filter=nb_filter, kernel_size=kernel_size,padding='same')
    
    #need convolution on shortcut for add different channel
    if with_conv_shortcut:
        shortcut = Conv3d_BN(input_model,nb_filter=nb_filter,strides=strides,kernel_size=kernel_size)
        x = layers.add([x,shortcut])
        return x
    else:
        x = layers.add([x,input_model])
        return x

In [ ]:
# from keras.layers.wrappers import Bidirectional, TimeDistributed
# help(TimeDistributed)

In [ ]:
class MODEL(object):
    def __init__(self, img_c=3, img_w=100, img_h=50, cord=40, frames_n=77, frames_cord=76, output_size=len(tok.word_index)):
        self.img_c = img_c
        self.img_w = img_w
        self.img_h = img_h
        self.cord = cord
        self.frames_n = frames_n
        self.frames_cord = frames_cord
        self.output_size = output_size
        self.build()
    
    def build(self):
        self.input_data = Input(name='the_input', shape=(77,50,100,3), dtype='float32')
        
        self.zero1 = ZeroPadding3D(padding=(1, 2, 2), name='zero1')(self.input_data)
        self.conv1 = Conv3D(64, (3, 5, 5), strides=(1, 2, 2), kernel_initializer='he_normal', name='conv1')(self.zero1)
        self.batc1 = BatchNormalization(name='batc1')(self.conv1)
        self.actv1 = Activation('relu', name='actv1')(self.batc1)
        self.drop1 = SpatialDropout3D(0)(self.actv1)
        self.maxp1 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')(self.drop1)

#         self.zero2 = ZeroPadding3D(padding=(1, 2, 2), name='zero2')(self.maxp1)
#         self.conv2 = Conv3D(64, (3, 5, 5), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv2')(self.zero2)
#         self.batc2 = BatchNormalization(name='batc2')(self.conv2)
#         self.actv2 = Activation('relu', name='actv2')(self.batc2)
#         self.drop2 = SpatialDropout3D(0)(self.actv2)
#         self.maxp2 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')(self.drop2)

#         self.zero3 = ZeroPadding3D(padding=(1, 1, 1), name='zero3')(self.maxp2)
#         self.conv3 = Conv3D(96, (3, 3, 3), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv3')(self.zero3)
#         self.batc3 = BatchNormalization(name='batc3')(self.conv3)
#         self.actv3 = Activation('relu', name='actv3')(self.batc3)
#         self.drop3 = SpatialDropout3D(0)(self.actv3)
#         self.maxp3 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')(self.drop3)

        #Residual conv2_x ouput 56x56x64 
        self.x = Residual_Block(self.maxp1,nb_filter=64,kernel_size=(3,3,3))
        self.x = Residual_Block(self.x,nb_filter=64,kernel_size=(3,3,3))
        self.x = Residual_Block(self.x,nb_filter=64,kernel_size=(3,3,3))
        
        #Residual conv3_x ouput 28x28x128 
        self.x = Residual_Block(self.x,nb_filter=128,kernel_size=(3,3,3),strides=(2,2,2),with_conv_shortcut=True)# need do convolution to add different channel
        self.x = Residual_Block(self.x,nb_filter=128,kernel_size=(3,3,3))
        self.x = Residual_Block(self.x,nb_filter=128,kernel_size=(3,3,3))
        self.x = Residual_Block(self.x,nb_filter=128,kernel_size=(3,3,3))
        
        #Residual conv4_x ouput 14x14x256
        self.x = Residual_Block(self.x,nb_filter=256,kernel_size=(3,3,3),strides=(2,2,2),with_conv_shortcut=True)# need do convolution to add different channel
        self.x = Residual_Block(self.x,nb_filter=256,kernel_size=(3,3,3))
        self.x = Residual_Block(self.x,nb_filter=256,kernel_size=(3,3,3))
        self.x = Residual_Block(self.x,nb_filter=256,kernel_size=(3,3,3))
        self.x = Residual_Block(self.x,nb_filter=256,kernel_size=(3,3,3))
        self.x = Residual_Block(self.x,nb_filter=256,kernel_size=(3,3,3))
        
        #Residual conv5_x ouput 7x7x512
        self.x = Residual_Block(self.x,nb_filter=512,kernel_size=(3,3,3),strides=(2,2,2),with_conv_shortcut=True)
        self.x = Residual_Block(self.x,nb_filter=512,kernel_size=(3,3,3))
        self.x = Residual_Block(self.x,nb_filter=512,kernel_size=(3,3,3))
        
        #Using AveragePooling replace flatten
        self.x = AveragePooling3D()(self.x)

        self.resh1 = TimeDistributed(Flatten())(self.x)

        self.gru_1 = GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1')(self.resh1)
        self.drop4 = Dropout(0)(self.gru_1)
        self.gru_2 = GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru2')(self.drop4)
        self.drop5 = Dropout(0)(self.gru_2)

        self.resh2 = Flatten()(self.drop5)
        
        self.input_cord = Input(name='the_input_cord', shape=(76,40), dtype='float32')
        self.gru_3 = GRU(32, return_sequences=True, kernel_initializer='Orthogonal', name='gru3')(self.input_cord)
        self.drop6 = Dropout(0)(self.gru_3)
        self.gru_4 = GRU(32, return_sequences=True, kernel_initializer='Orthogonal', name='gru4')(self.drop6)
        self.drop7 = Dropout(0)(self.gru_4)
        
        self.resh3 = Flatten()(self.drop7)
        
        self.z = Model(inputs = self.input_data, outputs = self.resh2) #
        self.y = Model(inputs = self.input_cord, outputs = self.resh3) #
        self.combined = layers.concatenate([self.z.output, self.y.output])
        
        # transforms RNN output to character activations:
        self.dense1 = Dense(self.output_size, kernel_initializer='he_normal', name='dense1')(self.combined)

        self.y_pred = Activation('softmax', name='softmax')(self.dense1)

        self.model = Model(inputs = [self.input_data, self.input_cord], outputs = self.y_pred)
        
    def summary(self):
        Model(inputs=[self.input_data, self.input_cord], outputs=self.y_pred).summary()

In [ ]:
# output_size = 200
model = MODEL(img_c=3,img_w=100,img_h=50,cord=40,frames_n=77,frames_cord=76,output_size=len(tok.word_index))
model.summary()

In [ ]:
# 恢复模型结构及权重
# model.model.load_weights('./weight/CnnGRU_bright_BestWeights.h5')

In [ ]:
# adam = Adam(lr=0.00001)
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# def categorical_crossentropy(target, output, from_logits=False, axis=-1):
#     """Categorical crossentropy between an output tensor and a target tensor.
#     Arguments:
#       target: A tensor of the same shape as `output`.
#       output: A tensor resulting from a softmax
#           (unless `from_logits` is True, in which
#           case `output` is expected to be the logits).
#       from_logits: Boolean, whether `output` is the
#           result of a softmax, or is a tensor of logits.
#       axis: Int specifying the channels axis. `axis=-1` corresponds to data
#           format `channels_last', and `axis=1` corresponds to data format
#           `channels_first`.
#     Returns:
#       Output tensor.
#     Raises:
#       ValueError: if `axis` is neither -1 nor one of the axes of `output`.
#     """

#     rank = len(output.shape)
#     axis = axis % rank
#     # Note: nn.softmax_cross_entropy_with_logits_v2
#     # expects logits, Keras expects probabilities.
#     if not from_logits:
#     # scale preds so that the class probas of each sample sum to 1
#     output = output / math_ops.reduce_sum(output, axis, True)
#     # manual computation of crossentropy
#     epsilon_ = _to_tensor(epsilon(), output.dtype.base_dtype)
#     output = clip_ops.clip_by_value(output, epsilon_, 1. - epsilon_)
#     return -math_ops.reduce_sum(target * math_ops.log(output), axis)
#     else:
#     return nn.softmax_cross_entropy_with_logits_v2(labels=target, logits=output)

In [ ]:
--------------------------------------------------------------------------------------------------------------

In [ ]:
# checkpoint
filepath="./weight/ResnetGRU_cord_dif_GRU_32_normal.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
history = model.model.fit(x = [image_train, cord_train],
                          y = label_train, 
                          validation_data = ([image_val, cord_val],label_val), 
                          epochs = 100, 
                          batch_size = minibatch_size, 
                          callbacks = [checkpoint], 
                          shuffle = True, 
                          verbose = 2)

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel('train')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='center right')
    plt.show()

In [ ]:
show_train_history(history, "acc", "val_acc")

In [ ]:
--------------------------------------------------------------------------------------------------------------

# print中間層的output結果

In [ ]:
#取某一层的输出为输出新建为model，采用函数模型
layer_model = Model(inputs = model.input_data, outputs = model.model.get_layer('max1').output)

#以这个model的预测值作为输出
middle_output = layer_model.predict(image_train)
print(middle_output.shape)

In [ ]:
print(middle_output)

In [ ]:
--------------------------------------------------------------------------------------------------------------

In [ ]:
# 保存模型结构及权重
# model.model.save('./weight/CnnGRU_50_vocab.h5')

# predict

In [ ]:
# 恢复模型结构及权重
model.model.load_weights('./weight/ResnetGRU_cord_dif_GRU_32_normal.h5')

In [ ]:
predict = model.model.predict([image_val, cord_val])           # predict result is value
# temp = model.model.predict_classes(image_val)     # predict class, but only use model = Sequential() format
predict = np.argmax(predict,axis=1)
# predict

In [ ]:
result = []
for i in range(len(predict)):
    temp = labels_to_text([predict[i]])
    result.append(temp)
# print(result)

In [ ]:
# for i in result:
#     print('Predict label:', i)

In [ ]:
# for i in txt_val:
#     print('True label:', i)

# evaluate

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_names = split_word
print(classification_report(txt_val, result, target_names=target_names))

In [ ]:
train_acc = model.model.evaluate(image_train, label_train)
print ('\ntrain Acc:', train_acc[1])

val_acc = model.model.evaluate(image_val, label_val)
print ('\nTest Acc:', val_acc[1])